In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# %matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [ ]:
import copy
import pygmt
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams["text.usetex"] = False

In [ ]:
COMMAND_FILE_NAME = "../data/config/western_north_america_config.json"
model = celeri.build_model(COMMAND_FILE_NAME)

In [ ]:
# Plot TDE displacements from a single element
tri_idx = 0
vel_east, vel_north, vel_up = celeri.get_tri_displacements(
    model.station.lon.to_numpy(),
    model.station.lat.to_numpy(),
    model.meshes,
    model.config.material_lambda,
    model.config.material_mu,
    tri_idx=0,
    strike_slip=1,
    dip_slip=0,
    tensile_slip=0,
)
plt.figure()
plt.plot(model.meshes[0].lon1[tri_idx], model.meshes[0].lat1[tri_idx], ".r")
plt.plot(model.meshes[0].lon2[tri_idx], model.meshes[0].lat2[tri_idx], ".r")
plt.plot(model.meshes[0].lon3[tri_idx], model.meshes[0].lat3[tri_idx], ".r")
plt.quiver(model.station.lon, model.station.lat, vel_east, vel_north, scale=1e-3)
plt.show()

In [ ]:
# Create a plot with coast, Miller projection (J) over the continental US
min_lon = -135.0
max_lon = -110.0
min_lat = 30.0
max_lat = 50.0
region = [min_lon, max_lon, min_lat, max_lat]
topo_data = "@earth_relief_30s"
# topo_data = '@earth_relief_01m'
projection = "J-65/12c"

fig = pygmt.Figure()
pygmt.config(MAP_FRAME_TYPE="plain")
pygmt.config(MAP_FRAME_PEN="0.25p")
pygmt.config(MAP_TICK_PEN="0.25p")

pygmt.makecpt(
    cmap="topo",
    # cmap='terra',
    # series='-4000/4000/20',
    # series='-11000/4000/500',
    continuous=True,
)

fig.grdimage(
    grid=topo_data,
    region=region,
    # shading=True,
    projection=projection,
)

fig.coast(
    region=region,
    projection=projection,
    # land="lightgray",
    # water="white",
    area_thresh=4000,
    shorelines="0.25p,200/200/200",
    frame="p",
)

# Plot block boundaries
for i in range(model.closure.n_polygons()):
    fig.plot(
        x=model.closure.polygons[i].vertices[:, 0],
        y=model.closure.polygons[i].vertices[:, 1],
        pen="0.25p,0/0/0",
    )

# Plot GPS stations
# fig.plot(x=station.lon, y=station.lat, style="c0.05", color="yellow", pen="0.1p,black")
fig.show()

In [ ]:
# Stained glass block polygons
rng = np.random.default_rng(42)

fig = pygmt.Figure()
pygmt.config(MAP_FRAME_TYPE="plain")
pygmt.config(MAP_FRAME_PEN="0.25p")
pygmt.config(MAP_TICK_PEN="0.25p")

pygmt.makecpt(cmap="gray", series="-4000/4000/20", continuous=True)

fig.grdimage(
    grid=topo_data,
    region=region,
    shading=True,
    projection=projection,
)

# Plot blocks as transparent polygons
for i in range(model.closure.n_polygons()):
    red = 100 + 50 * rng.uniform()
    green = 150 + 100 * rng.uniform()
    blue = 150 + 100 * rng.uniform()
    color_string = str(red) + "/" + str(green) + "/" + str(blue) + "@75"
    fig.plot(
        x=model.closure.polygons[i].vertices[:, 0],
        y=model.closure.polygons[i].vertices[:, 1],
        fill=color_string,
    )

# Plot block boundaries
for i in range(model.closure.n_polygons()):
    fig.plot(
        x=model.closure.polygons[i].vertices[:, 0],
        y=model.closure.polygons[i].vertices[:, 1],
        pen="0.1p,0/0/0",
    )

fig.coast(
    region=region,
    projection=projection,
    area_thresh=4000,
    shorelines="0.25p,255/255/255",
    frame="p",
)

fig.show()